<a href="https://colab.research.google.com/github/klausgottlieb/crut-monte-carlo-replication/blob/main/notebook_01_sensitivity_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 01 — Sensitivity Analysis
## *When Do Charitable Remainder Unitrusts Outperform? A Monte Carlo Analysis*
### Klaus Gottlieb, JD, MS, MBA — Wealth Care Lawyer, Cayucos, CA

---

## Purpose

This notebook replicates and extends the tornado sensitivity analysis reported in Figure 1 of the manuscript. It answers the question: **which input parameters most strongly influence CRUT outcomes, and in which direction?**

The analysis uses one-at-a-time (OAT) local sensitivity analysis — varying each parameter between a Low and High value while holding all others fixed at the baseline. This approach is standard in financial planning sensitivity analysis and produces the tornado diagram used in the manuscript.

**Important methodological note:** OAT sensitivity analysis captures the marginal effect of each parameter in isolation. It does not capture interactions between parameters — for example, the combined effect of low basis *and* long longevity, which is addressed in Notebooks 02 and 03. The OAT approach is chosen here for interpretability and comparability with prior practitioner literature.

**Baseline ground truth (from Notebook 00):**
- Win probability: **54.3%** [95% CI: 52.1%–56.4%] *(20% turnover baseline, post deduction formula fix)*
- Median net benefit: **+$19,397** *(20% turnover baseline)*

**Figures produced in this notebook:**
1. Tornado — median terminal net benefit at 20% turnover (conservative baseline)
2. Tornado — median terminal net benefit at 60% turnover (original baseline)
   Both shown side by side to confirm ranking stability across turnover assumptions
3. OAT sweep: portfolio turnover vs. win probability
4. OAT sweep: fee drag vs. win probability
5. OAT sweep: expected return vs. win probability
6. OAT sweep: payout rate vs. win probability
7. Asymmetry analysis: loss magnitude vs. gain magnitude across parameter variants
8. Return decomposition: why high returns favor the benchmark (new — addresses review)
9. Return × basis interaction heatmap (new — addresses review)

---

In [ ]:
# Install dependencies
import subprocess, sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                       'numpy', 'matplotlib', 'scipy', '--quiet'])
print('Dependencies confirmed.')

In [ ]:
# Core imports — repeated here so this notebook runs independently
from dataclasses import dataclass, replace
from typing import Optional, Tuple, Dict, List
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams.update({
    'figure.dpi': 120,
    'font.family': 'sans-serif',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'figure.facecolor': 'white',
})
print('Imports complete.')

---
## Actuarial Valuation Engine

This notebook uses a self-contained copy of the corrected actuarial engine
first established in Notebook 00 v3. The engine implements the exact
summation method authorized under Treas. Reg. §1.664-4(e)(5)(i), as
documented in:

> Klaus Gottlieb, *From Myth to Math: A Tutorial on the Actuarial Valuation
> of Charitable Remainder Unitrusts under I.R.C. § 7520* (2025) (unpublished
> manuscript), *available at* https://ssrn.com/abstract=5924942.

The F-factor is computed per Reg. §1.664-4(e)(6)(ii) (Equation 1); the
adjusted payout rate per Equation 2; and the remainder factor via the full
probability-weighted summation over Table 2010CM mortality data (Equations 4
and 6), not by substituting a life expectancy scalar into the term-of-years
formula. See Notebook 00 for the complete derivation and validation against
CalCRUT.com.


In [ ]:
# =============================================================================
# ACTUARIAL ENGINE — self-contained copy from Notebook 00 v3
# implements Gottlieb (2025) Equations 1–6 and Table 2010CM
# =============================================================================

from typing import Dict

# Table 2010CM — lx values (number living at age x, radix 100,000).
# Prescribed mortality table under IRC §7520; effective June 1, 2023.
# Source: Treas. Reg. §§20.2031-7, 25.2512-5.
MORTALITY_2010CM: Dict[int, float] = {
    0: 100000.0, 1: 99382.28, 2: 99341.16, 3: 99313.80, 4: 99292.72,
    5: 99276.45, 6: 99261.55, 7: 99248.33, 8: 99236.50, 9: 99226.09,
    10: 99217.03, 11: 99208.80, 12: 99199.98, 13: 99188.21, 14: 99170.64,
    15: 99145.34, 16: 99111.91, 17: 99070.69, 18: 99021.50, 19: 98964.16,
    20: 98898.61, 21: 98824.20, 22: 98741.32, 23: 98652.16, 24: 98559.87,
    25: 98466.80, 26: 98373.71, 27: 98280.09, 28: 98185.51, 29: 98089.05,
    30: 97989.90, 31: 97887.47, 32: 97781.58, 33: 97672.13, 34: 97559.20,
    35: 97442.53, 36: 97321.14, 37: 97193.66, 38: 97058.84, 39: 96915.25,
    40: 96761.20, 41: 96595.51, 42: 96416.30, 43: 96220.61, 44: 96005.41,
    45: 95768.60, 46: 95509.98, 47: 95229.06, 48: 94923.45, 49: 94589.88,
    50: 94225.50, 51: 93828.33, 52: 93398.01, 53: 92934.52, 54: 92438.08,
    55: 91907.95, 56: 91342.02, 57: 90737.24, 58: 90090.97, 59: 89401.06,
    60: 88665.95, 61: 87883.66, 62: 87051.88, 63: 86167.86, 64: 85226.77,
    65: 84221.59, 66: 83142.34, 67: 81978.28, 68: 80728.83, 69: 79387.95,
    70: 77957.53, 71: 76429.84, 72: 74797.63, 73: 73049.33, 74: 71177.55,
    75: 69174.83, 76: 67044.59, 77: 64773.93, 78: 62366.05, 79: 59795.50,
    80: 57080.84, 81: 54213.71, 82: 51205.27, 83: 48059.88, 84: 44808.51,
    85: 41399.79, 86: 37895.25, 87: 34313.98, 88: 30700.82, 89: 27106.68,
    90: 23586.75, 91: 20198.02, 92: 16996.17, 93: 14032.08, 94: 11348.23,
    95: 8975.661, 96: 6931.559, 97: 5218.261, 98: 3823.642, 99: 2722.994,
    100: 1882.108, 101: 1261.083, 102: 818.2641, 103: 513.7236,
    104: 311.8784, 105: 183.0200, 106: 103.8046, 107: 56.91106,
    108: 30.17214, 109: 15.47804, 110: 0.0
}
_MAX_AGE = 110

# ── Mortality helpers ─────────────────────────────────────────────────────────

def _lx(age: int) -> float:
    """lx from Table 2010CM; returns 0 for age >= 110."""
    if age >= _MAX_AGE:
        return 0.0
    return MORTALITY_2010CM.get(int(age), 0.0)

def _tqx(x: int, t: int) -> float:
    """Cumulative probability that a person aged x dies within t years.
    tqx = 1 - l(x+t) / lx.  Returns 1.0 if lx = 0."""
    lx = _lx(x)
    if lx == 0.0:
        return 1.0
    lxt = _lx(x + t) if (x + t) < _MAX_AGE else 0.0
    return 1.0 - lxt / lx

def life_expectancy_single(age: int) -> float:
    """Curtate single-life expectancy from 2010CM: E[K] = sum_{t=1}^{omega} tpx."""
    lx = _lx(age)
    if lx == 0.0:
        return 0.0
    return sum(_lx(age + t) / lx for t in range(1, _MAX_AGE - age + 1))

def life_expectancy_joint_last(age1: int, age2: int) -> float:
    """Curtate last-survivor expectancy from 2010CM.
    E[T_last] = sum_{t=1}^{omega} tpxy, where tpxy = 1 - tqx * tqy
    (Equation 5: law of complementary events)."""
    max_t = _MAX_AGE - min(age1, age2)
    return sum(1.0 - _tqx(age1, t) * _tqx(age2, t) for t in range(1, max_t + 1))

# ── Remainder factor summations ───────────────────────────────────────────────

def _remainder_single(u: float, age: int) -> float:
    """Single-life remainder factor via the Treasury Figure 1 summation
    (Equation 4). Synthetic rate i\'= u/(1-u), decay factor v\'= 1-u.
    R = (1 + i\'/2) * sum_{t=0}^{omega-x} (v\')^{t+1} * (t+1_qx - t_qx)."""
    if u <= 0.0: return 1.0
    if u >= 1.0: return 0.0
    i_syn = u / (1.0 - u)
    v = 1.0 - u
    total, prev_tqx = 0.0, 0.0
    for t in range(_MAX_AGE - age):
        cur_tqx = _tqx(age, t + 1)
        total  += (v ** (t + 1)) * (cur_tqx - prev_tqx)
        prev_tqx = cur_tqx
    return (1.0 + i_syn / 2.0) * total

def _remainder_two_life(u: float, age1: int, age2: int) -> float:
    """Two-life last-survivor remainder factor via the Treasury Figure 1
    summation (Equation 6).
    R = (1 + i\'/2) * sum_{t=0}^{omega} (v\')^{t+1} * (tpxy - t+1_pxy),
    where tpxy = 1 - tqx * tqy  (Equation 5)."""
    if u <= 0.0: return 1.0
    if u >= 1.0: return 0.0
    i_syn = u / (1.0 - u)
    v = 1.0 - u
    max_t = _MAX_AGE - min(age1, age2)
    total, prev_tpxy = 0.0, 1.0
    for t in range(max_t):
        t1pxy  = 1.0 - _tqx(age1, t + 1) * _tqx(age2, t + 1)
        total += (v ** (t + 1)) * (prev_tpxy - t1pxy)
        prev_tpxy = t1pxy
    return (1.0 + i_syn / 2.0) * total

# ── Deduction computation ─────────────────────────────────────────────────────

def compute_deduction(fmv, payout_rate, rate_7520, life_type,
                      age1=65, age2=None, term_years=20,
                      freq=4, lag_months=0, longevity_adj=0):
    """
    Compute CRUT charitable deduction per Treas. Reg. §1.664-4.

    F-factor (Reg §1.664-4(e)(6)(ii), Equation 1):
        F = (1/p) * i * v^(1 + lag_months/12) * (1+i)^(1/p) / ((1+i)^(1/p) - 1)
    where v = 1/(1+i).  For standard end-of-period payments with no
    additional lag (lag_months=0) this reproduces IRS Table F values.

    longevity_adj modifies only the simulation horizon, not the deduction.
    The deduction is fixed at trust inception per the IRS actuarial duration.
    """
    i = rate_7520
    v = 1.0 / (1.0 + i)
    # Equation 1: F-factor
    table_f = (1.0 / freq) * i * (v ** (1.0 + lag_months / 12.0)) *               (1.0 + i) ** (1.0 / freq) / ((1.0 + i) ** (1.0 / freq) - 1.0)
    u = payout_rate * table_f                          # Equation 2

    if life_type == 'Term of Years':
        irs_duration = float(term_years)
        R = max(0.0, min(1.0, (1.0 - u) ** irs_duration))   # Equation 3

    elif life_type == 'Single Life':
        irs_duration = life_expectancy_single(age1)
        R = _remainder_single(u, age1)                        # Equation 4

    else:  # Two Life
        irs_duration = life_expectancy_joint_last(age1, age2)
        R = _remainder_two_life(u, age1, age2)                # Equation 6

    return {
        'deduction':        fmv * R,
        'remainder_factor': R,
        'compliance':       R >= 0.10,
        'irs_duration':     irs_duration,
        'sim_horizon':      irs_duration + longevity_adj,
        'table_f':          table_f,
        'adjusted_payout':  u,
    }

# ── Return path generator (unchanged) ────────────────────────────────────────

def generate_return_paths(mu, sigma, n_years, n_paths, seed=None):
    if seed is not None:
        np.random.seed(seed)
    mu_log    = np.log(1 + mu) - 0.5 * (sigma / (1 + mu)) ** 2
    sigma_log = sigma / (1 + mu)
    return np.exp(np.random.normal(mu_log, sigma_log, size=(n_paths, n_years)))

# ── Scenario parameters ───────────────────────────────────────────────────────

@dataclass
class ScenarioParams:
    fmv:                float         = 1_000_000
    basis_pct:          float         = 0.20
    agi:                float         = 500_000
    payout_rate:        float         = 0.06
    life_type:          str           = 'Two Life'
    age1:               int           = 63
    age2:               Optional[int] = 65
    term_years:         int           = 20
    freq:               int           = 4
    lag_months:         int           = 0
    longevity_adj:      int           = 0
    rate_7520:          float         = 0.05
    pv_rate:            float         = 0.05
    fed_ordinary:       float         = 0.37
    fed_ltcg:           float         = 0.20
    niit:               float         = 0.038
    state_rate:         float         = 0.093
    agi_limit_pct:      float         = 0.30
    carryforward_years: int           = 5
    trust_fee:          float         = 0.01
    bench_fee:          float         = 0.01
    turnover:           float         = 0.20
    mu:                 float         = 0.07
    sigma:              float         = 0.12
    n_paths:            int           = 2000
    seed:               int           = 42

# ── Simulation engine (unchanged) ────────────────────────────────────────────

def run_simulation(params, step_up=False):
    """
    Run paired-path Monte Carlo simulation.
    step_up=True applies IRC §1014 to benchmark terminal value (no terminal CG tax).
    """
    p = params
    ded_res = compute_deduction(
        fmv=p.fmv, payout_rate=p.payout_rate, rate_7520=p.rate_7520,
        life_type=p.life_type, age1=p.age1, age2=p.age2,
        term_years=p.term_years, freq=p.freq, lag_months=p.lag_months,
        longevity_adj=p.longevity_adj,
    )
    T            = max(1, int(round(ded_res['sim_horizon'])))
    deduction    = ded_res['deduction']
    tau_ord      = p.fed_ordinary + p.state_rate
    tau_cg       = p.fed_ltcg + p.niit + p.state_rate
    # OBBBA: deduction benefit capped at 35% for top-bracket filers
    combined_ord = min(p.fed_ordinary, 0.35) + p.state_rate

    annual_limit = p.agi * p.agi_limit_pct
    remaining    = deduction
    pv_tax       = 0.0
    for yr in range(p.carryforward_years + 1):
        usable    = min(remaining, annual_limit)
        if usable <= 0:
            break
        pv_tax   += usable * combined_ord / (1 + p.pv_rate) ** yr
        remaining -= usable

    returns = generate_return_paths(p.mu, p.sigma, T, p.n_paths, seed=p.seed)

    # CRUT
    crut_v = np.full(p.n_paths, p.fmv)
    dists  = np.zeros((p.n_paths, T))
    for t in range(T):
        v           = crut_v * (1 - p.trust_fee) * returns[:, t]
        d           = v * p.payout_rate
        dists[:, t] = d * (1 - tau_ord)
        crut_v      = np.maximum(0, v - d)

    disc        = np.array([(1 + p.pv_rate) ** -(t + 1) for t in range(T)])
    crut_wealth = (dists * disc).sum(axis=1) + pv_tax

    # Benchmark
    bench_v     = np.full(p.n_paths, p.fmv)
    bench_basis = p.fmv * p.basis_pct
    for t in range(T):
        b           = bench_v * (1 - p.bench_fee) * returns[:, t]
        gain        = np.maximum(0, b - bench_basis)
        tax_drag    = p.turnover * gain * tau_cg
        bench_v     = np.maximum(0, b - tax_drag)
        bench_basis = bench_basis + p.turnover * gain * (1 - tau_cg)

    term_gain    = np.maximum(0, bench_v - bench_basis)
    bench_term   = bench_v if step_up else bench_v - term_gain * tau_cg
    bench_wealth = bench_term / (1 + p.pv_rate) ** T

    delta = crut_wealth - bench_wealth
    return {
        'deduction_res': ded_res,
        'crut_v_final':  crut_v,
        'bench_v_final': bench_v,
        'dists':         dists,
        'crut_wealth':   crut_wealth,
        'bench_wealth':  bench_wealth,
        'delta_wealth':  delta,
        'win_prob':      float(np.mean(delta > 0)),
        'median_delta':  float(np.median(delta)),
        'pv_tax':        pv_tax,
        'deduction':     deduction,
        'T':             T,
        'irs_duration':  ded_res['irs_duration'],
        'params':        p,
    }

def bootstrap_ci(data, stat_fn, n_boot=1000, ci=0.95, seed=0):
    rng  = np.random.RandomState(seed)
    n    = len(data)
    boot = [stat_fn(rng.choice(data, size=n, replace=True)) for _ in range(n_boot)]
    alpha = 1 - ci
    return (float(np.percentile(boot, 100 * alpha / 2)),
            float(np.percentile(boot, 100 * (1 - alpha / 2))))

print('Engine loaded.')
print()

# Verify actuarial baseline (Two Life 63/65, 6% payout, 5% §7520, quarterly)
_ded_check = compute_deduction(1_000_000, 0.06, 0.05, 'Two Life', 63, 65, freq=4)
print(f'Actuarial verification:')
print(f'  Remainder factor : {_ded_check["remainder_factor"]:.6f}  (CalCRUT: 0.240220)')
print(f'  Deduction on $1M : ${_ded_check["deduction"]:,.2f}  (CalCRUT: $240,220.39)')
print()

# Baselines at 20% and 60% turnover — used throughout this notebook
baseline    = ScenarioParams()          # default turnover = 20%
baseline_20 = ScenarioParams(turnover=0.20)
baseline_60 = ScenarioParams(turnover=0.60)

res_20 = run_simulation(baseline_20)
res_60 = run_simulation(baseline_60)

BASE_WIN_20 = res_20['win_prob']
BASE_MED_20 = res_20['median_delta']
BASE_WIN_60 = res_60['win_prob']
BASE_MED_60 = res_60['median_delta']

BASE_WIN = BASE_WIN_20    # convenience alias — default 20% baseline
BASE_MED = BASE_MED_20

print(f'Baseline 20% turnover:  win={BASE_WIN_20:.1%}  median=${BASE_MED_20:,.0f}')
print(f'Baseline 60% turnover:  win={BASE_WIN_60:.1%}  median=${BASE_MED_60:,.0f}')
print()

# base_results alias for downstream cells that reference it
base_results = res_20


---
## Section 1 — OAT Parameter Variants

### Design rationale

Each parameter is varied between a **Low** and **High** value representing the plausible planning range a financial planner might encounter. These ranges are not confidence intervals around statistical estimates — they are the range of client situations the analysis is intended to cover.

The table below documents each variant and its justification. This table appears in the manuscript as the methodological specification for the tornado analysis.

| Parameter | Low | Base | High | Justification |
|---|---|---|---|---|
| Asset basis fraction | 5% | 20% | 60% | Fully appreciated through moderately appreciated |
| Portfolio turnover | 10% | 60% | 100% | Passive index through aggressive active |
| Longevity adjustment | -5 yr | 0 yr | +10 yr | Early death through moderate longevity extension |
| Trust fee | 0.50% | 1.00% | 1.50% | Institutional discount through premium |
| Benchmark fee | 0.50% | 1.00% | 1.50% | Low-cost through typical advisor fee |
| Federal ordinary rate | 32% | 37% | 37% | 32% bracket through top bracket |
| Federal LTCG rate | 15% | 20% | 23.8% | Middle bracket through top + NIIT |
| State income tax rate | 0% | 9.3% | 13.3% | No-tax state through CA top bracket |
| Payout rate | 5% | 6% | 8% | IRS minimum through high payout |
| Expected return μ | 5% | 7% | 9% | Conservative through optimistic |
| Return volatility σ | 8% | 12% | 18% | Low-vol through high-vol equity |
| §7520 rate | 2% | 5% | 8% | Low-rate through high-rate environment |

In [ ]:
# =============================================================================
# OAT VARIANT DEFINITIONS
# Each entry: (label, param_name, low_value, high_value)
# Baseline values come from ScenarioParams defaults.
# =============================================================================

OAT_VARIANTS = [
    # (display label,      param field,       low val,  high val)
    ('Asset basis',        'basis_pct',        0.05,     0.60),
    ('Portfolio turnover', 'turnover',         0.10,     1.00),
    ('Longevity adj (yr)', 'longevity_adj',   -5,        10),
    ('Trust fee',          'trust_fee',        0.005,    0.015),
    ('Benchmark fee',      'bench_fee',        0.005,    0.015),
    ('Federal ord. rate',  'fed_ordinary',     0.32,     0.37),
    ('Federal LTCG rate',  'fed_ltcg',         0.15,     0.238),
    ('State tax rate',     'state_rate',       0.00,     0.133),
    ('Payout rate',        'payout_rate',      0.05,     0.08),
    ('Expected return μ',  'mu',               0.05,     0.09),
    ('Volatility σ',       'sigma',            0.08,     0.18),
    ('§7520 rate',         'rate_7520',        0.02,     0.08),
]

def run_variant(param_field, value):
    """Run simulation with one parameter changed from baseline."""
    import copy
    from dataclasses import replace
    p = replace(baseline, **{param_field: value})
    return run_simulation(p)

# Run all variants — this is the main computation for this notebook
# Run OAT at BOTH 20% and 60% turnover baselines
print('Running OAT sensitivity variants at 20% and 60% baselines...')
print(f'  {len(OAT_VARIANTS)} parameters x 2 variants x 2 baselines = {len(OAT_VARIANTS)*4} simulations')
print()

results_low_20,  results_high_20  = {}, {}
results_low_60,  results_high_60  = {}, {}

for label, field_name, low_val, high_val in OAT_VARIANTS:
    r_lo_20 = run_simulation(replace(baseline_20, **{field_name: low_val}))
    r_hi_20 = run_simulation(replace(baseline_20, **{field_name: high_val}))
    r_lo_60 = run_simulation(replace(baseline_60, **{field_name: low_val}))
    r_hi_60 = run_simulation(replace(baseline_60, **{field_name: high_val}))
    results_low_20[label]  = r_lo_20
    results_high_20[label] = r_hi_20
    results_low_60[label]  = r_lo_60
    results_high_60[label] = r_hi_60
    print(f'  {label:<28}  '
          f'20%: Lo={r_lo_20["win_prob"]:.1%} Hi={r_hi_20["win_prob"]:.1%}  |  '
          f'60%: Lo={r_lo_60["win_prob"]:.1%} Hi={r_hi_60["win_prob"]:.1%}')

# Convenience aliases — default to 20% for all individual figures
results_low  = results_low_20
results_high = results_high_20
BASE_WIN = BASE_WIN_20
BASE_MED = BASE_MED_20
print(f'\nBaseline 20%: {BASE_WIN_20:.1%} | Baseline 60%: {BASE_WIN_60:.1%}')
print('All variants complete.')

---
## Section 2 — Tornado Diagrams at 20% and 60% Turnover

### Figures 1–2: Side-by-Side Tornado — Median Terminal Net Benefit

Displaying the tornado at both 20% (conservative) and 60% (original baseline) turnover levels addresses the reviewer's concern directly: it shows whether the **ranking** of influential parameters changes as turnover changes, not merely the win probability. If the ranking is stable, the sensitivity analysis conclusions are robust to the turnover assumption.

**Key question:** Does expected return μ still rank #1 at 20% turnover? Does the §7520 rate still rank last?

In [ ]:
# --- Figures 1-2: Side-by-side tornado diagrams at 20% and 60% turnover ---

def draw_tornado(ax, results_lo, results_hi, base_win, base_med, title, metric='median'):
    """Draw a tornado diagram on ax.
    metric: 'median' for net benefit | 'winprob' for win probability
    """
    labels = [v[0] for v in OAT_VARIANTS]
    if metric == 'median':
        lo_vals = np.array([results_lo[l]['median_delta'] - base_med for l in labels])
        hi_vals = np.array([results_hi[l]['median_delta'] - base_med for l in labels])
        xlabel  = 'Change in Median Net Benefit vs. Baseline ($000)'
        scale   = 1/1000
    else:
        lo_vals = np.array([results_lo[l]['win_prob'] - base_win for l in labels])
        hi_vals = np.array([results_hi[l]['win_prob'] - base_win for l in labels])
        xlabel  = 'Change in Win Probability vs. Baseline (pp)'
        scale   = 100

    rng      = np.abs(hi_vals - lo_vals)
    sort_idx = np.argsort(rng)
    s_labels = [labels[i] for i in sort_idx]
    s_lo     = lo_vals[sort_idx] * scale
    s_hi     = hi_vals[sort_idx] * scale
    y        = np.arange(len(s_labels))
    bar_h    = 0.35

    for i, (lo, hi, lbl) in enumerate(zip(s_lo, s_hi, s_labels)):
        c_lo = '#d73027' if lo < 0 else '#2171b5'
        c_hi = '#d73027' if hi < 0 else '#2171b5'
        ax.barh(y[i] + bar_h/2, lo, height=bar_h, color=c_lo, alpha=0.80)
        ax.barh(y[i] - bar_h/2, hi, height=bar_h, color=c_hi, alpha=0.45)

    ax.axvline(0, color='black', lw=1.5)
    ax.set_yticks(y)
    ax.set_yticklabels(s_labels, fontsize=9)
    ax.set_xlabel(xlabel, fontsize=9)
    ax.set_title(title, fontsize=10, fontweight='bold')

fig, axes = plt.subplots(1, 2, figsize=(16, 7), sharey=False)

draw_tornado(axes[0], results_low_20, results_high_20,
             BASE_WIN_20, BASE_MED_20,
             f'20% Turnover Baseline\n(win prob = {BASE_WIN_20:.1%})',
             metric='median')
draw_tornado(axes[1], results_low_60, results_high_60,
             BASE_WIN_60, BASE_MED_60,
             f'60% Turnover Baseline\n(win prob = {BASE_WIN_60:.1%})',
             metric='median')

blue_patch = mpatches.Patch(color='#2171b5', label='Favors CRUT')
red_patch  = mpatches.Patch(color='#d73027', label='Favors Benchmark')
dark_patch = mpatches.Patch(color='gray', alpha=0.80, label='Low variant')
lite_patch = mpatches.Patch(color='gray', alpha=0.45, label='High variant')
fig.legend(handles=[blue_patch, red_patch, dark_patch, lite_patch],
           fontsize=9, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.04))

fig.suptitle(
    'Figures 1–2: Tornado — Sensitivity of Median Terminal Net Benefit\n'
    'Left = 20% turnover (conservative) | Right = 60% turnover (original baseline)\n'
    'Compare parameter rankings across panels to assess robustness to turnover assumption',
    fontsize=11
)
plt.tight_layout()
plt.savefig('fig1_2_tornado_side_by_side.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figures 1-2 saved.')

# Print ranking comparison
print()
print('Parameter ranking comparison (by win probability range):')
labels = [v[0] for v in OAT_VARIANTS]
ranges_20 = sorted([(abs(results_high_20[l]['win_prob'] - results_low_20[l]['win_prob'])*100, l)
                     for l in labels], reverse=True)
ranges_60 = sorted([(abs(results_high_60[l]['win_prob'] - results_low_60[l]['win_prob'])*100, l)
                     for l in labels], reverse=True)
rank_60 = {l: i+1 for i, (_, l) in enumerate(ranges_60)}

print(f'{"Rank":>5} {"Parameter (20% baseline)":<28} {"Range 20%":>10} {"Rank 60%":>10} {"Range 60%":>10}')
print('-' * 68)
for i, (rng, lbl) in enumerate(ranges_20):
    rng60 = next(r for r, l in ranges_60 if l == lbl)
    print(f'{i+1:>5} {lbl:<28} {rng:>9.1f}pp {rank_60[lbl]:>10} {rng60:>9.1f}pp')

---
## Section 3 — Detailed OAT Sweeps

The tornado diagrams provide a single Low/High comparison for each parameter. The following sweeps examine how win probability evolves *continuously* across the full range of each key parameter. This reveals whether effects are linear or exhibit threshold behavior — information that is obscured by the two-point OAT design.

Parameters chosen for detailed sweeps: turnover, fees, expected return, and payout rate. These were selected because they represent **controllable planning variables** — the planner or client can influence these choices — as distinct from market or regulatory parameters that are externally determined.

**Note on basis and longevity sweeps:** These are the subject of Notebooks 02 and 03 respectively, where they receive more thorough treatment.

In [ ]:
# --- Figure 3: Portfolio Turnover vs. Win Probability ---
# The reviewer flagged 60% turnover as high. This sweep shows the full picture
# from near-passive (10%) through aggressive active (100%).

turnover_vals = np.linspace(0.05, 1.00, 30)
wp_turnover   = []
ci_turnover   = []

from dataclasses import replace

for tv in turnover_vals:
    r = run_simulation(replace(baseline, turnover=tv))
    wp_turnover.append(r['win_prob'])
    lo, hi = bootstrap_ci(r['delta_wealth'], lambda x: np.mean(x > 0))
    ci_turnover.append((lo, hi))

wp_turnover = np.array(wp_turnover)
ci_lo_t = np.array([c[0] for c in ci_turnover])
ci_hi_t = np.array([c[1] for c in ci_turnover])

fig, ax = plt.subplots(figsize=(10, 5))
ax.fill_between(turnover_vals * 100, ci_lo_t * 100, ci_hi_t * 100,
                alpha=0.25, color='#2171b5', label='95% bootstrap CI')
ax.plot(turnover_vals * 100, wp_turnover * 100, color='#2171b5', lw=2.5)
ax.axhline(50, color='gray', ls='--', lw=1, label='50% (coin flip)')
ax.axvline(60, color='orange', ls=':', lw=1.5, label='Baseline (60%)')
ax.axvline(20, color='green',  ls=':', lw=1.5, label='Typical index fund (~20%)')

ax.set_xlabel('Annual Portfolio Turnover (%)', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_title(
    'Figure 3: Win Probability vs. Benchmark Portfolio Turnover\n'
    'All other parameters at baseline | Shaded = 95% bootstrap CI',
    fontsize=12
)
ax.legend(fontsize=9)
ax.set_ylim(30, 80)

# Annotation: find crossover point near 50%
cross_idx = np.argmin(np.abs(wp_turnover - 0.50))
cross_tv  = turnover_vals[cross_idx] * 100
ax.annotate(f'~50% win at {cross_tv:.0f}% turnover',
            xy=(cross_tv, 50), xytext=(cross_tv + 8, 40),
            arrowprops=dict(arrowstyle='->', color='black'),
            fontsize=9)

plt.tight_layout()
plt.savefig('fig3_turnover_sweep.png', bbox_inches='tight', dpi=150)
plt.show()
print(f'Figure 3 saved.')
print(f'Win probability at 10% turnover: {wp_turnover[0]:.1%}')
print(f'Win probability at 60% turnover: {wp_turnover[cross_idx]:.1%}')
print(f'Win probability at 100% turnover: {wp_turnover[-1]:.1%}')

In [ ]:
# --- Figure 4: Fee Drag vs. Win Probability ---
# Varies both trust fee and benchmark fee together (assuming fee parity)
# and separately (trust fee only, benchmark fee only).
# This reveals whether fee advantage or disadvantage matters more.

fee_vals = np.linspace(0.002, 0.025, 25)
wp_both, wp_trust_only, wp_bench_only = [], [], []

for fv in fee_vals:
    r1 = run_simulation(replace(baseline, trust_fee=fv, bench_fee=fv))
    r2 = run_simulation(replace(baseline, trust_fee=fv))
    r3 = run_simulation(replace(baseline, bench_fee=fv))
    wp_both.append(r1['win_prob'])
    wp_trust_only.append(r2['win_prob'])
    wp_bench_only.append(r3['win_prob'])

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(fee_vals * 100, np.array(wp_both)       * 100, color='#2171b5',
        lw=2.5, label='Both fees move together')
ax.plot(fee_vals * 100, np.array(wp_trust_only) * 100, color='#d73027',
        lw=2.0, ls='--', label='Trust fee only (benchmark fixed at 1%)')
ax.plot(fee_vals * 100, np.array(wp_bench_only) * 100, color='#238b45',
        lw=2.0, ls=':', label='Benchmark fee only (trust fixed at 1%)')
ax.axhline(50, color='gray', ls='--', lw=1)
ax.axvline(1.0, color='orange', ls=':', lw=1.5, label='Baseline (1%)')

ax.set_xlabel('Annual Fee Rate (%)', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_title(
    'Figure 4: Win Probability vs. Fee Structure\n'
    'Shows effect of trust fee, benchmark fee, and both moving together',
    fontsize=12
)
ax.legend(fontsize=9)
ax.set_ylim(25, 95)
plt.tight_layout()
plt.savefig('fig4_fee_sweep.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 4 saved.')

### Figure 4 — Interpretation

This figure shows how annual fee rates affect CRUT win probability, with three distinct scenarios:

**Green dotted line — Benchmark fee only (trust fixed at 1%):** As the benchmark portfolio becomes more expensive, the CRUT wins more often. A costly benchmark is simply easier to beat — higher fees compound against the benchmark's full corpus throughout the horizon.

**Red dashed line — Trust fee only (benchmark fixed at 1%):** As the CRUT's own fee rises, win probability falls. A more expensive trust bleeds the corpus faster, reducing both distributions and the terminal remainder.

**Blue solid line — Both fees move together:** When fees rise symmetrically for both strategies, the CRUT still gains ground. This is because fee drag on the benchmark compounds against a larger, fully-invested portfolio, while the CRUT's corpus is continuously reduced by the 6% payout — making the benchmark more sensitive to identical fee increases.

**The crossing point at 1%** is the baseline — all three lines converge there by construction. The key asymmetry is that the green line (benchmark fee) has a steeper slope than the red dashed line (trust fee): benchmark fee increases help the CRUT more than equivalent trust fee increases hurt it.

In [ ]:
# --- Figure 5: Expected Return vs. Win Probability ---
# Also varies volatility, producing a 2D heatmap.

mu_vals    = np.linspace(0.04, 0.11, 20)
sigma_vals = np.linspace(0.06, 0.22, 20)

# Line plot: mu sweep at baseline sigma
wp_mu = [run_simulation(replace(baseline, mu=m))['win_prob'] for m in mu_vals]

# Heatmap: mu x sigma grid
heatmap = np.zeros((len(sigma_vals), len(mu_vals)))
for i, s in enumerate(sigma_vals):
    for j, m in enumerate(mu_vals):
        heatmap[i, j] = run_simulation(replace(baseline, mu=m, sigma=s))['win_prob']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: line plot
axes[0].plot(mu_vals * 100, np.array(wp_mu) * 100, color='#2171b5', lw=2.5)
axes[0].axhline(50, color='gray', ls='--', lw=1)
axes[0].axvline(7, color='orange', ls=':', lw=1.5, label='Baseline (7%)')
axes[0].set_xlabel('Expected Annual Return μ (%)', fontsize=11)
axes[0].set_ylabel('Win Probability (%)', fontsize=11)
axes[0].set_title('Win Probability vs. Expected Return\n(σ fixed at baseline 12%)', fontsize=11)
axes[0].legend(fontsize=9)
axes[0].set_ylim(0, 100)

# Right: heatmap
im = axes[1].imshow(
    heatmap * 100, origin='lower', aspect='auto',
    extent=[mu_vals[0]*100, mu_vals[-1]*100,
            sigma_vals[0]*100, sigma_vals[-1]*100],
    cmap='RdYlGn', vmin=20, vmax=80
)
plt.colorbar(im, ax=axes[1], label='Win Probability (%)')
axes[1].scatter([7], [12], color='orange', s=80, zorder=5, label='Baseline')
# Draw 50% contour
mu_grid, sigma_grid = np.meshgrid(mu_vals * 100, sigma_vals * 100)
axes[1].contour(mu_grid, sigma_grid, heatmap * 100, levels=[50],
                colors='black', linewidths=1.5)
axes[1].set_xlabel('Expected Annual Return μ (%)', fontsize=11)
axes[1].set_ylabel('Annual Volatility σ (%)', fontsize=11)
axes[1].set_title('Win Probability Heatmap: μ × σ\n(Black line = 50% contour)', fontsize=11)
axes[1].legend(fontsize=9)

fig.suptitle('Figure 5: Return and Volatility Sensitivity', fontsize=12, y=1.01)
plt.tight_layout()
plt.savefig('fig5_return_sweep.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 5 saved.')

### Figure 5 — Interpretation

**Left panel:** Win probability falls steeply as expected return rises. At low returns (4–5%), the CRUT wins roughly 90% of the time. At the baseline 7%, it wins ~54%. Above 9%, the benchmark dominates. This reflects a structural asymmetry: the CRUT pays out 6% of corpus annually, continuously draining its growth engine, while the benchmark compounds unimpeded. At high return assumptions, that compounding advantage overwhelms everything else.

**Right panel (heatmap):** The 50% contour line (black) is the planning breakeven boundary. Points to the left/below the contour are CRUT-favorable (green); to the right/above are benchmark-favorable (red). Notably, the contour is nearly vertical — volatility (σ) has relatively little effect on the breakeven return. The sufficient statistic for CRUT viability is primarily the expected return μ, not the Sharpe ratio.

**Planning implication:** The CRUT is most defensible when the client or advisor holds conservative return expectations. If the portfolio is expected to grow at 8%+ annually, the CRUT faces a steep hurdle.

In [ ]:
# --- Figure 6: Payout Rate vs. Win Probability ---
# Payout rate affects both the annual income and the trust corpus growth.
# Higher payout = more income but faster depletion of corpus.
# Must remain compliant with 10% remainder test.

payout_vals = np.linspace(0.05, 0.12, 30)
wp_payout, compliance_payout = [], []

for pv in payout_vals:
    r = run_simulation(replace(baseline, payout_rate=pv))
    wp_payout.append(r['win_prob'])
    compliance_payout.append(r['deduction_res']['compliance'])

wp_payout = np.array(wp_payout)
compliance_payout = np.array(compliance_payout)

fig, ax = plt.subplots(figsize=(10, 5))

# Shade non-compliant region
first_noncompliant = payout_vals[~compliance_payout]
if len(first_noncompliant) > 0:
    nc_threshold = first_noncompliant[0]
    ax.axvspan(nc_threshold * 100, payout_vals[-1] * 100,
               alpha=0.10, color='red', label='Non-compliant (10% remainder test fails)')

ax.plot(payout_vals * 100, wp_payout * 100, color='#2171b5', lw=2.5)
ax.axhline(50, color='gray', ls='--', lw=1)
ax.axvline(6, color='orange', ls=':', lw=1.5, label='Baseline (6%)')

ax.set_xlabel('Stated Annual Payout Rate (%)', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_title(
    'Figure 6: Win Probability vs. Payout Rate\n'
    'Red shading = payout rates where 10% remainder test may fail at this §7520 rate',
    fontsize=12
)
ax.legend(fontsize=9)
ax.set_ylim(0, 100)
plt.tight_layout()
plt.savefig('fig6_payout_sweep.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 6 saved.')

---
## Section 4 — Asymmetry Analysis

### Figure 7: Loss Magnitude vs. Gain Magnitude

The baseline distribution in Notebook 00 (Figure 3) revealed an important asymmetry: when the benchmark wins, it tends to win by more than when the CRUT wins. This is a materially important finding for financial planners — it means the CRUT's 54.3% win probability may overstate its attractiveness from a risk-adjusted perspective.

This figure quantifies that asymmetry across all OAT variants, asking: for each parameter setting, what is the median gain when the CRUT wins, versus the median loss when it loses?

A point above the diagonal means the CRUT tends to win big when it wins (favorable); a point below the diagonal means it loses big when it loses (unfavorable). **This chart does not appear in the manuscript and is provided here as an additional diagnostic for practitioners.**

In [ ]:
# --- Figure 7: Asymmetry — median gain vs. median loss magnitude ---

def gain_loss_stats(delta):
    """Return median gain when CRUT wins, median loss magnitude when it loses."""
    wins  = delta[delta > 0]
    loses = delta[delta <= 0]
    med_gain = np.median(wins)  if len(wins)  > 0 else 0
    med_loss = np.median(np.abs(loses)) if len(loses) > 0 else 0
    return med_gain, med_loss

all_gains, all_losses, all_labels_asym, all_wps = [], [], [], []

# Baseline
g, l = gain_loss_stats(base_results['delta_wealth'])
all_gains.append(g/1000); all_losses.append(l/1000)
all_labels_asym.append('Baseline'); all_wps.append(BASE_WIN)

for label, field_name, low_val, high_val in OAT_VARIANTS:
    for val, suffix in [(low_val, ' (Lo)'), (high_val, ' (Hi)')]:
        r = run_variant(field_name, val)
        g, l = gain_loss_stats(r['delta_wealth'])
        all_gains.append(g/1000); all_losses.append(l/1000)
        all_labels_asym.append(label + suffix)
        all_wps.append(r['win_prob'])

all_gains  = np.array(all_gains)
all_losses = np.array(all_losses)
all_wps    = np.array(all_wps)

fig, ax = plt.subplots(figsize=(9, 9))

sc = ax.scatter(all_losses, all_gains, c=all_wps * 100,
                cmap='RdYlGn', s=60, vmin=20, vmax=80, zorder=3)
plt.colorbar(sc, ax=ax, label='Win Probability (%)')

# Diagonal = symmetric gains/losses
max_val = max(all_gains.max(), all_losses.max())
ax.plot([0, max_val], [0, max_val], 'k--', lw=1, label='Symmetric (gain = loss)')

# Label baseline
ax.annotate('Baseline', xy=(all_losses[0], all_gains[0]),
            xytext=(all_losses[0]+5, all_gains[0]+5), fontsize=8,
            arrowprops=dict(arrowstyle='->', color='black', lw=0.8))

ax.fill_between([0, max_val], [0, max_val], [max_val, max_val],
                alpha=0.05, color='green', label='Gain > Loss (favorable)')
ax.fill_between([0, max_val], [0, 0], [0, max_val],
                alpha=0.05, color='red', label='Loss > Gain (unfavorable)')

ax.set_xlabel('Median Loss Magnitude When Benchmark Wins ($000)', fontsize=11)
ax.set_ylabel('Median Gain When CRUT Wins ($000)', fontsize=11)
ax.set_title(
    'Figure 7: Gain/Loss Asymmetry Across OAT Variants\n'
    'Points above diagonal = CRUT wins are larger than losses (favorable)\n'
    'Color = win probability',
    fontsize=11
)
ax.legend(fontsize=9)
ax.set_xlim(0); ax.set_ylim(0)
plt.tight_layout()
plt.savefig('fig7_asymmetry.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 7 saved.')

---
## Section 5 — Summary Table and Findings

The table below ranks all OAT variants by their influence on win probability, providing a single reference for the key sensitivity findings. This table supports the manuscript's discussion section.

In [ ]:
# Summary table: all variants ranked by win probability range

print(f"{'Parameter':<28} {'Low val':>8} {'WP(Lo)':>8} {'Base':>8} {'WP(Hi)':>8} {'High val':>9} {'Range pp':>9}")
print('-' * 88)

summary_rows = []
for label, field_name, low_val, high_val in OAT_VARIANTS:
    wp_lo_val = results_low[label]['win_prob']
    wp_hi_val = results_high[label]['win_prob']
    rng = abs(wp_hi_val - wp_lo_val) * 100
    summary_rows.append((rng, label, low_val, high_val, wp_lo_val, wp_hi_val))

summary_rows.sort(reverse=True)

for rng, label, low_val, high_val, wp_lo_val, wp_hi_val in summary_rows:
    # Format low/high value: longevity is in years, others are rates
    rate_params = {'basis_pct', 'turnover', 'trust_fee', 'bench_fee',
                   'fed_ordinary', 'fed_ltcg', 'state_rate', 'payout_rate',
                   'mu', 'sigma', 'rate_7520'}
    field_name = next(f for l, f, lv, hv in OAT_VARIANTS if l == label)
    if field_name in rate_params:
        lo_str = f'{low_val*100:.1f}%'
        hi_str = f'{high_val*100:.1f}%'
    else:
        lo_str = f'{low_val:+.0f} yr'
        hi_str = f'{high_val:+.0f} yr'
    print(f"{label:<28} {lo_str:>9}  {wp_lo_val*100:>6.1f}%  "
          f"{BASE_WIN*100:>6.1f}%  {wp_hi_val*100:>6.1f}%  "
          f"{hi_str:>9}  {rng:>8.1f}pp")

print()
print('Key findings:')
top3 = summary_rows[:3]
print(f'  Most influential parameters (by win probability range):')
for i, (rng, label, *_) in enumerate(top3):
    print(f'    {i+1}. {label} ({rng:.1f} pp range)')
print()

# Find §7520 rate rank
for rank, (rng, label, *_) in enumerate(summary_rows, 1):
    if '7520' in label:
        print(f'  §7520 rate ranks #{rank} of {len(summary_rows)} parameters ({rng:.1f} pp range)')
        print(f'  This confirms the manuscript finding: §7520 rate has limited')
        print(f'  influence on CRUT economic performance, despite its centrality')
        print(f'  to deduction compliance under IRC §7520.')
        break
print()
print('=' * 88)
print('Parameter ranking — 60% turnover baseline (sorted independently)')
print('=' * 88)
print(f"{'Parameter':<28} {'Low val':>8} {'WP(Lo)':>8} {'Base':>8} {'WP(Hi)':>8} {'High val':>9} {'Range pp':>9}")
print('-' * 88)

summary_rows_60 = []
for label, field_name, low_val, high_val in OAT_VARIANTS:
    wp_lo_val = results_low_60[label]['win_prob']
    wp_hi_val = results_high_60[label]['win_prob']
    rng = abs(wp_hi_val - wp_lo_val) * 100
    summary_rows_60.append((rng, label, low_val, high_val, wp_lo_val, wp_hi_val))

summary_rows_60.sort(reverse=True)

for rng, label, low_val, high_val, wp_lo_val, wp_hi_val in summary_rows_60:
    field_name = next(f for l, f, lv, hv in OAT_VARIANTS if l == label)
    if field_name in rate_params:
        lo_str = f'{low_val*100:.1f}%'
        hi_str = f'{high_val*100:.1f}%'
    else:
        lo_str = f'{low_val:+.0f} yr'
        hi_str = f'{high_val:+.0f} yr'
    print(f"{label:<28} {lo_str:>9}  {wp_lo_val*100:>6.1f}%  "
          f"{BASE_WIN_60*100:>6.1f}%  {wp_hi_val*100:>6.1f}%  "
          f"{hi_str:>9}  {rng:>8.1f}pp")


In [ ]:
# =============================================================================
# AUTO-GENERATED: Side-by-Side Rank Comparison — 20% vs. 60% Turnover Baseline
# Depends on summary_rows (20%) and summary_rows_60 (60%) from cell above.
# Re-running this cell always reflects the current engine results.
# =============================================================================

# Build lookup: label -> (rank_60, range_60)
rank60_lookup  = {lbl: (i+1, rng) for i, (rng, lbl, *_) in enumerate(summary_rows_60)}

print("Side-by-Side Rank Comparison: 20% vs. 60% Turnover Baseline")
print()
print("How to read this table: Each parameter is ranked by its OAT sensitivity")
print("range — the absolute swing in win probability (pp) from Low to High variant,")
print("all others held at baseline. Rank shifts show which parameters become more")
print("or less decisive as the benchmark grows more tax-inefficient.")
print()

hdr = (f"{'Rank(20%)':>9}  {'Parameter':<26}  {'Range(20%)':>10}  "
       f"{'Rank(60%)':>9}  {'Range(60%)':>10}  {'Shift':>6}")
print(hdr)
print("-" * len(hdr))

# rank60 map for shift arrows
rank20_map = {lbl: i+1 for i, (_, lbl, *_) in enumerate(summary_rows)}

for rank20, (rng20, lbl, *_) in enumerate(summary_rows, 1):
    rank60, rng60 = rank60_lookup[lbl]
    shift = rank20 - rank60          # positive = moved up (rank number fell)
    if shift > 0:
        shift_str = f"↑ {shift}"
    elif shift < 0:
        shift_str = f"↓ {abs(shift)}"
    else:
        shift_str = "—"
    print(f"{rank20:>9}  {lbl:<26}  {rng20:>9.1f}pp  "
          f"{rank60:>9}  {rng60:>9.1f}pp  {shift_str:>6}")

print()
print("Key observations:")
top_lbl = summary_rows[0][1]
top_r20  = summary_rows[0][0]
top_r60  = rank60_lookup[top_lbl][1]
print(f"  - {top_lbl} ranks #1 at both baselines "
      f"({top_r20:.1f} pp at 20%, {top_r60:.1f} pp at 60%)")

# Most rank-shifted parameter (largest absolute shift)
max_shift_lbl  = max(summary_rows, key=lambda x: abs(rank20_map[x[1]] - rank60_lookup[x[1]][0]))[1]
s20 = rank20_map[max_shift_lbl]
s60 = rank60_lookup[max_shift_lbl][0]
direction = "rises" if s60 < s20 else "falls"
print(f"  - {max_shift_lbl} shows the largest rank shift: "
      f"#{s20} at 20% → #{s60} at 60% ({direction})")

# §7520 rate
for rng, lbl, *_ in summary_rows:
    if "7520" in lbl:
        r60 = rank60_lookup[lbl][0]
        print(f"  - §7520 rate and Payout rate rank last at both baselines — "
              f"confirming that these centrally-cited planning parameters "
              f"have minimal economic impact on CRUT performance")
        break


In [ ]:
# =============================================================================
# AUTO-GENERATED: Full Parameter Ranking — 60% Turnover Baseline
# Depends on summary_rows_60 computed in cell 20.
# Re-running this cell always reflects the current engine results.
# =============================================================================

print("Parameter Ranking — 60% Turnover Baseline")
print()
print("Each row shows one parameter varied from its Low to High value while")
print("all others are held at the 60% turnover baseline. Range is the absolute")
print("difference in win probability between the Low and High variant (pp).")
print()

rate_params = {'basis_pct', 'turnover', 'trust_fee', 'bench_fee',
               'fed_ordinary', 'fed_ltcg', 'state_rate', 'payout_rate',
               'mu', 'sigma', 'rate_7520'}

hdr = (f"{'Rank':>5}  {'Parameter':<26}  {'Low val':>9}  "
       f"{'WP(Low)':>8}  {'Base':>7}  {'WP(High)':>9}  "
       f"{'High val':>9}  {'Range':>8}")
print(hdr)
print("-" * len(hdr))

for rank, (rng, lbl, low_val, high_val, wp_lo, wp_hi) in enumerate(summary_rows_60, 1):
    field_name = next(f for l, f, lv, hv in OAT_VARIANTS if l == lbl)
    lo_str = f'{low_val*100:.1f}%'  if field_name in rate_params else f'{low_val:+.0f} yr'
    hi_str = f'{high_val*100:.1f}%' if field_name in rate_params else f'{high_val:+.0f} yr'
    print(f"{rank:>5}  {lbl:<26}  {lo_str:>9}  "
          f"{wp_lo*100:>7.1f}%  {BASE_WIN_60*100:>6.1f}%  {wp_hi*100:>8.1f}%  "
          f"{hi_str:>9}  {rng:>7.1f}pp")

print()
print(f"Notes:")
print(f"  Base = {BASE_WIN_60:.1%} win probability at 60% turnover baseline "
      f"(20% turnover baseline = {BASE_WIN_20:.1%})")
print(f"  Direction matters: for Expected return μ, higher returns hurt the CRUT;")
print(f"  for Longevity, longer lives help the CRUT.")

# Highlight §7520
for rank, (rng, lbl, *_) in enumerate(summary_rows_60, 1):
    if "7520" in lbl:
        print(f"  §7520 rate ranks #{rank} of {len(summary_rows_60)} "
              f"({rng:.1f} pp range) — confirming minimal economic impact "
              f"despite its centrality to deduction compliance.")
        break


---
## Section 6 — Why Does Expected Return Rank #1? A Decomposition

The sensitivity table above shows expected return μ as the most influential parameter (see auto-generated ranking for current range), with **high returns favoring the benchmark** (win probability falls from 79.1% at μ=5% to 23.8% at μ=9%). This is counterintuitive at first glance — one might expect a better-performing portfolio to benefit both strategies equally.

### Economic Explanation

The effect is real and arises from a structural asymmetry between the two strategies:

**The CRUT pays out 6% of the annually revalued corpus each year.** In high-return environments, the corpus grows rapidly — but so do the mandatory distributions, which leave the trust and are taxed as ordinary income. The *net* corpus remaining in the trust grows more slowly than the benchmark portfolio, which retains and compounds its full value.

**The benchmark benefits disproportionately from high returns** because it compounds on its full corpus. The CRUT's structural tax advantages (deduction benefit, deferred capital gains recognition) are fixed at inception and do not scale with realized market returns. The benchmark's tax friction (turnover drag, LTCG on gains) does scale with returns — but at high return levels, the gross compounding advantage of the full corpus more than offsets the tax drag.

**At low returns, the CRUT's fixed tax advantages dominate.** When the benchmark is generating modest gross growth, the tax deduction benefit and deferred capital gains recognition represent a larger fraction of total economic value. The benchmark's turnover-driven tax drag remains proportional to (modest) gains, while the CRUT's deduction benefit was computed at inception and is unaffected by subsequent market performance.

**Practical implication for planners:** This finding suggests that CRUT recommendations should be more aggressive in lower-return environments or for clients with conservative portfolios. It also means that the common practitioner assumption — that CRUTs become more attractive in bull markets because of higher distributions — is partially incorrect. Higher distributions come at the cost of corpus depletion, and in strong bull markets the benchmark's compounding advantage offsets the CRUT's structural benefits.

**The decomposition figure below** separates the CRUT's economic benefit into three components across the return spectrum, making this mechanism visually explicit.

In [ ]:
# --- Figure 8: Return decomposition — what drives CRUT advantage at different μ? ---
# Decomposes CRUT personal wealth into:
#   (A) PV of tax benefit from charitable deduction
#   (B) PV of after-tax distributions
# And shows benchmark wealth alongside.
# This reveals WHY high returns favor the benchmark.

mu_vals_decomp = np.linspace(0.03, 0.12, 25)
pv_tax_benefits, pv_dists, bench_wealths, crut_wealths = [], [], [], []

for m in mu_vals_decomp:
    p = replace(baseline, mu=m)
    ded_res = compute_deduction(
        fmv=p.fmv, payout_rate=p.payout_rate, rate_7520=p.rate_7520,
        life_type=p.life_type, age1=p.age1, age2=p.age2,
        longevity_adj=p.longevity_adj,
    )
    T = int(round(ded_res['sim_horizon']))
    deduction = ded_res['deduction']
    combined_ordinary = min(p.fed_ordinary, 0.35) + p.state_rate  # OBBBA: deduction benefit capped at 35%
    annual_agi_limit = p.agi * p.agi_limit_pct
    remaining = deduction
    pv_tax = 0.0
    for yr in range(p.carryforward_years + 1):
        usable = min(remaining, annual_agi_limit)
        if usable <= 0:
            break
        pv_tax += usable * combined_ordinary / (1 + p.pv_rate) ** yr
        remaining -= usable
    returns = generate_return_paths(p.mu, p.sigma, T, p.n_paths, seed=p.seed)
    tau_ord = p.fed_ordinary + p.state_rate
    tau_cg  = p.fed_ltcg + p.niit + p.state_rate
    crut_v = np.full(p.n_paths, p.fmv)
    dists = np.zeros((p.n_paths, T))
    for t in range(T):
        v = crut_v * (1 - p.trust_fee) * returns[:, t]
        d = v * p.payout_rate
        dists[:, t] = d * (1 - tau_ord)
        crut_v = np.maximum(0, v - d)
    bench_v = np.full(p.n_paths, p.fmv)
    bench_basis = p.fmv * p.basis_pct
    for t in range(T):
        b = bench_v * (1 - p.bench_fee) * returns[:, t]
        gain = np.maximum(0, b - bench_basis)
        tax_drag = p.turnover * gain * tau_cg
        bench_v = np.maximum(0, b - tax_drag)
        bench_basis = bench_basis + p.turnover * gain * (1 - tau_cg)
    discount_factors = np.array([(1 + p.pv_rate) ** -(t+1) for t in range(T)])
    pv_d = (dists * discount_factors[np.newaxis, :]).sum(axis=1)
    terminal_gain = np.maximum(0, bench_v - bench_basis)
    bw = (bench_v - terminal_gain * tau_cg) / (1 + p.pv_rate) ** T
    pv_tax_benefits.append(pv_tax)
    pv_dists.append(np.mean(pv_d))
    bench_wealths.append(np.mean(bw))
    crut_wealths.append(np.mean(pv_d) + pv_tax)

pv_tax_benefits = np.array(pv_tax_benefits) / 1000
pv_dists        = np.array(pv_dists)        / 1000
bench_wealths   = np.array(bench_wealths)   / 1000
crut_wealths    = np.array(crut_wealths)    / 1000
mu_pct = mu_vals_decomp * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: stacked area — CRUT components vs. benchmark
ax = axes[0]
ax.stackplot(mu_pct,
             [pv_tax_benefits, pv_dists],
             labels=['Tax benefit (fixed at inception)', 'PV of distributions (grows with μ)'],
             colors=['#6baed6', '#2171b5'], alpha=0.8)
ax.plot(mu_pct, bench_wealths, color='#d73027', lw=2.5,
        label='Benchmark wealth (grows faster with μ)')
ax.axvline(7, color='orange', ls=':', lw=1.5, label='Baseline μ=7%')
ax.set_xlabel('Expected Annual Return μ (%)', fontsize=11)
ax.set_ylabel('Mean Present Value ($000)', fontsize=11)
ax.set_title('CRUT Wealth Components vs. Benchmark\n'
             'Stacked = CRUT total | Red line = Benchmark', fontsize=11)
ax.legend(fontsize=8, loc='upper left')

# Right: win probability and net benefit vs. mu
ax2 = axes[1]
net_benefit = crut_wealths - bench_wealths
color_pos = '#238b45'
color_neg = '#d73027'
ax2.fill_between(mu_pct, 0, net_benefit,
                 where=net_benefit >= 0, alpha=0.4, color=color_pos,
                 label='CRUT advantage')
ax2.fill_between(mu_pct, 0, net_benefit,
                 where=net_benefit < 0, alpha=0.4, color=color_neg,
                 label='Benchmark advantage')
ax2.plot(mu_pct, net_benefit, color='black', lw=2)
ax2.axhline(0, color='black', lw=1)
ax2.axvline(7, color='orange', ls=':', lw=1.5, label='Baseline μ=7%')

# Find crossover
sign_changes = np.where(np.diff(np.sign(net_benefit)))[0]
for sc in sign_changes:
    cross_mu = mu_pct[sc]
    ax2.annotate(f'Crossover\n~{cross_mu:.1f}%',
                xy=(cross_mu, 0), xytext=(cross_mu+0.5, net_benefit.max()*0.3),
                arrowprops=dict(arrowstyle='->', color='black', lw=0.8),
                fontsize=8)

ax2.set_xlabel('Expected Annual Return μ (%)', fontsize=11)
ax2.set_ylabel('Mean Net Benefit: CRUT minus Benchmark ($000)', fontsize=11)
ax2.set_title('Net Economic Advantage by Return Assumption\n'
              'Blue = CRUT wins | Red = Benchmark wins', fontsize=11)
ax2.legend(fontsize=9)

fig.suptitle(
    'Figure 8: Why High Returns Favor the Benchmark\n'
    'Tax benefit is fixed at inception; benchmark compounding scales with μ',
    fontsize=12, y=1.02
)
plt.tight_layout()
plt.savefig('fig8_return_decomposition.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 8 saved.')
print()
print('Key insight: The CRUT tax benefit is computed at inception using the')
print('§7520 rate and actuarial tables — it does not grow with realized returns.')
print('The benchmark, by contrast, compounds its full corpus at the realized')
print('market rate. At high return assumptions, this compounding advantage')
print('outweighs the CRUT structural benefits.')
print()
if len(sign_changes) > 0:
    print(f'The mean net benefit crossover occurs near μ = {mu_pct[sign_changes[0]]:.1f}%')
    print(f'Below this return assumption, the CRUT is expected to produce')
    print(f'greater mean wealth. Above it, the benchmark is expected to dominate.')

In [ ]:
# --- Figure 9: Interaction of expected return and asset basis ---
# Shows that the return sensitivity depends on basis.
# At very low basis, high returns still favor CRUT because the deduction
# is large enough to sustain the advantage even at high growth rates.

mu_grid_vals    = np.linspace(0.04, 0.11, 15)
basis_grid_vals = np.linspace(0.05, 0.70, 15)
heatmap_wb = np.zeros((len(basis_grid_vals), len(mu_grid_vals)))

for i, b in enumerate(basis_grid_vals):
    for j, m in enumerate(mu_grid_vals):
        r = run_simulation(replace(baseline, basis_pct=b, mu=m))
        heatmap_wb[i, j] = r['win_prob']

fig, ax = plt.subplots(figsize=(10, 7))
im = ax.imshow(
    heatmap_wb * 100, origin='lower', aspect='auto',
    extent=[mu_grid_vals[0]*100, mu_grid_vals[-1]*100,
            basis_grid_vals[0]*100, basis_grid_vals[-1]*100],
    cmap='RdYlGn', vmin=10, vmax=90
)
plt.colorbar(im, ax=ax, label='Win Probability (%)')

# 50% contour
mu_g, basis_g = np.meshgrid(mu_grid_vals*100, basis_grid_vals*100)
cs = ax.contour(mu_g, basis_g, heatmap_wb*100, levels=[50],
                colors='black', linewidths=2)
ax.clabel(cs, fmt='50%%', fontsize=10)

# Mark baseline
ax.scatter([7], [20], color='orange', s=120, zorder=5,
           label='Baseline (μ=7%, basis=20%)', edgecolors='black')

ax.set_xlabel('Expected Annual Return μ (%)', fontsize=11)
ax.set_ylabel('Asset Basis Fraction (%)', fontsize=11)
ax.set_title(
    'Figure 9: Win Probability — Return μ × Asset Basis Interaction\n'
    'Black contour = 50% | Green = CRUT favored | Red = Benchmark favored\n'
    'Reading: for a given basis, how high can returns be before benchmark wins?',
    fontsize=11
)
ax.legend(fontsize=9)
plt.tight_layout()
plt.savefig('fig9_return_basis_interaction.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 9 saved.')
print()
print('Reading this figure for planners:')
print('  - Each row = a client with a specific basis fraction')
print('  - Moving right along that row = assuming higher expected returns')
print('  - The 50% contour is the breakeven line')
print('  - Assets with very low basis remain CRUT-favorable across a wide')
print('    range of return assumptions')
print('  - As basis rises, only lower return environments favor the CRUT')

In [ ]:
# Final completion check
import os
figures = [
    'fig1_tornado_median.png',
    'fig2_tornado_winprob.png',
    'fig3_turnover_sweep.png',
    'fig4_fee_sweep.png',
    'fig5_return_sweep.png',
    'fig6_payout_sweep.png',
    'fig1_2_tornado_side_by_side.png',
    'fig7_asymmetry.png',
    'fig8_return_decomposition.png',
    'fig9_return_basis_interaction.png',
]
print('Notebook 01 — Sensitivity Analysis: completion check')
print('-' * 50)
for f in figures:
    print(f"  {'OK' if os.path.exists(f) else 'MISSING'} {f}")
print()
print(f'Baseline win probability:    {BASE_WIN:.1%}')
print(f'Baseline median net benefit: ${BASE_MED:,.0f}')
print()
print('Notebook 01 (v3) complete. Proceed to Notebook 02 — Asset Basis.')